In [15]:
from vpython import *
import numpy as np
import random as rand
import matplotlib.pyplot as plt

In [16]:
def calc_wall(j, i, dire):
    if dire == 1:
        rcol = axes[j].pos - (particles[j][i].pos + R * vec(1,0,0))
        Fhat = vec(-1,0,0)
        
    if dire == -1:
        rcol = axes[j].pos - (particles[j][i].pos - R * vec(1,0,0))
        Fhat = vec(1,0,0)
        
    if dire == 2:
        rcol = axes[j].pos - (particles[j][i].pos + R * vec(0,1,0))
        Fhat = vec(0,-1,0)
        
    if dire == -2:
        rcol = axes[j].pos - (particles[j][i].pos -R * vec(0,1,0))
        Fhat = vec(0,1,0)
        
    if dire == 3:
        rcol = axes[j].pos - (particles[j][i].pos + R * vec(0,0,1))
        Fhat = vec(0,0,-1)
        
    if dire == -3:
        rcol = axes[j].pos - (particles[j][i].pos - R * vec(0,0,1))
        Fhat = vec(0,0,1)
           
    Fmag = -(mag(rcol) * mag(omega[j])) * m / dt  #This is the force that would be required to drop the velocity to 0
                                                  #and send the particle back at the same speed in the other direction
    F = Fmag * Fhat
    tau = cross(rcol,F)
    dL = tau * dt
    Li = I * omega[j]
    Lf = Li + dL
    omegaf = Lf/I
    
    return(omegaf)



In [18]:
scene = canvas(title = "Diatoms in a Box")

R = .5e-10  #radius of an atom
m = 1.7e-27 #mass of an atom
L = 40 * R
thick = L/100 #Thickness and positions of wall
Q = 3 * R
k = 1.4e-23
T = 300
s =  np.sqrt(2*3/2*k*T/m) #I like this as an initial speed

N = 10 #number of particles

particles = []
molecules = []
vcm = []
omega = []
axes = []
I = 2 * m * R**2
omegaf = 0
k = []


#The next loop in the code creates 2 particles that are touching and coupled in an array. The molecules have a random
#position and axis. The loop also assigns the diatoms a random center of mass velocity and angular velocity

for i in range(0,N):
    if i == 0:
        atom1 = sphere(pos = L/2*vec(rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.cyan, make_trail = True, retain = 50)
        r2 = rand.random()*R
        r3 = rand.random()*R
        r4 = abs((2*R)**2 - r2**2 - r3**2)
        a = np.sqrt(r4)
        atom2 = sphere(pos = atom1.pos - vec(r2,a,r3), radius = R, color = color.cyan, make_trail = True, retain = 50)
        particles.append([atom1,atom2])
        rod = cylinder(pos = vec((particles[i][1].pos.x + particles[i][0].pos.x)/2,(particles[i][1].pos.y + particles[i][0].pos.y)/2,(particles[i][1].pos.z + particles[i][0].pos.z)/2), axis = particles[i][0].pos - particles[i][1].pos, radius = R/4, visible = False)
        axes.append(rod)
        vcm.append(s*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
        omega.append(360*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
    else:
        atom1 = sphere(pos = L/2*vec(rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.red)
        r2 = rand.random()*R
        r3 = rand.random()*R
        r4 = abs((2*R)**2 - r2**2 - r3**2)
        a = np.sqrt(r4)
        atom2 = sphere(pos = atom1.pos - vec(r2,a,r3), radius = R, color = color.red)
        particles.append([atom1,atom2])
        rod = cylinder(pos = vec((particles[i][1].pos.x + particles[i][0].pos.x)/2,(particles[i][1].pos.y + particles[i][0].pos.y)/2,(particles[i][1].pos.z + particles[i][0].pos.z)/2), axis = particles[i][0].pos - particles[i][1].pos, radius = R/4, visible = False)
        axes.append(rod)
        vcm.append(s*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
        omega.append(360*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1))))
    
#Creating the walls of the box

Lwall = box(pos = vec(-L/2, 0, 0), size = vec(thick, L, L), color=color.white)
Rwall = box(pos = vec(L/2, 0, 0), size = vec(thick, L, L), color=color.white)
Bwall = box(pos = vec(0, -L/2, 0), size = vec(L, thick, L), color=color.white)
Twall = box(pos = vec(0, L/2, 0), size = vec(L, thick, L), color=color.white)
Zwall = box(pos = vec(0, 0, -L/2), size = vec(L, L, thick), color=color.white)

#Adding in the motion and the time steps

dt = R/s/10
Nstep = 3e3
t = 0

scene.pause()

while t < dt * Nstep:
    rate(100)
    for j in range(N):
        
        axes[j].pos = axes[j].pos + vcm[j] * dt
        
        #Do the actual math to figure out what the change in axis and fix the check collisions with the walls with the new
        #rod
        
        dtheta = omega[j] * dt / R
        
        Rx = [[1, 0, 0], [0, np.cos(dtheta.x), -np.sin(dtheta.x)], [0, np.sin(dtheta.x), np.cos(dtheta.x)]]
        Ry = [[np.cos(dtheta.y), 0, np.sin(dtheta.y)], [0, 1, 0], [-np.sin(dtheta.y), 0, np.cos(dtheta.y)]]
        Rz = [[np.cos(dtheta.z), -np.sin(dtheta.z), 0], [np.sin(dtheta.z), np.cos(dtheta.z), 0], [0, 0, 1]]
        
        
        newaxis = np.array([axes[j].axis.x, axes[j].axis.y, axes[j].axis.z])
        newaxis = np.dot(Rx,newaxis)
        newaxis = np.dot(Ry,newaxis)
        newaxis = np.dot(Rz,newaxis)
        axes[j].axis.x = newaxis[0]
        axes[j].axis.y = newaxis[1]
        axes[j].axis.z = newaxis[2]
        
        particles[j][0].pos = axes[j].pos - axes[j].axis/2
        particles[j][1].pos = axes[j].pos + axes[j].axis/2
        
        
        
        
        #Checking the center of mass collisions with the walls this is being done separate from the individual particles
        
        #This is checking to see if the individual particles are hitting the walls separate to the center of mass
        
        for i in range(0,1):
            if particles[j][i].pos.x  > L/2 - R:
                d = 1
                omega[j] = calc_wall(j,i,d)
                
            if particles[j][i].pos.x  < -L/2 + R:
                d = -1
                omega[j] = calc_wall(j,i,d)
            
            if particles[j][i].pos.y > L/2 - R:
                d = 2
                omega[j] = calc_wall(j,i,d)
                
            if particles[j][i].pos.y < -L/2 + R:
                d = -2
                omega[j] = calc_wall(j,i,d)
                        
            if particles[j][i].pos.z > L/2 - R:
                d = 3
                omega[j] = calc_wall(j,i,d)
                
            if particles[j][i].pos.z < -L/2 + R:
                d = -3
                omega[j] = calc_wall(j,i,d)
                    
        #This is testing to see if the center of mass is colliding with the wall
        
        if axes[j].pos.x > L/2:
            vcm[j].x = -vcm[j].x
            
        if axes[j].pos.x < -L/2:
            vcm[j].x = abs(vcm[j].x)
            
        if axes[j].pos.y > L/2:
            vcm[j].y = -vcm[j].y
            
        if axes[j].pos.y < -L/2:
            vcm[j].y = abs(vcm[j].y)
            
        if axes[j].pos.z > L/2:
            vcm[j].z = -vcm[j].z
            
        if axes[j].pos.z < -L/2:
            vcm[j].z = abs(vcm[j].z)
            
    k.append(.5 * I * mag(omega[0])**2) 
        
    t = t + dt
    
    
#The simulation looks very weird and I think it is because there is no friction from the walls that would cause the
#motion that we would expect to see from diatoms in a box


#It is also glitching a bit from having the new omega be calculated multiple times in a row which could be fixed
#With some sort of timer that only lets particles hit certain walls once in n number of time steps


<IPython.core.display.Javascript object>

In [14]:
tlist = np.linspace(0,t,num = N)

In [13]:
tlist

array([0.00000000e+00, 5.50973165e-12])

In [10]:
help(np.linspace)

Help on function linspace in module numpy.core.function_base:

linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)
    Return evenly spaced numbers over a specified interval.
    
    Returns `num` evenly spaced samples, calculated over the
    interval [`start`, `stop`].
    
    The endpoint of the interval can optionally be excluded.
    
    Parameters
    ----------
    start : scalar
        The starting value of the sequence.
    stop : scalar
        The end value of the sequence, unless `endpoint` is set to False.
        In that case, the sequence consists of all but the last of ``num + 1``
        evenly spaced samples, so that `stop` is excluded.  Note that the step
        size changes when `endpoint` is False.
    num : int, optional
        Number of samples to generate. Default is 50. Must be non-negative.
    endpoint : bool, optional
        If True, `stop` is the last sample. Otherwise, it is not included.
        Default is True.
    retstep : bo